In [ ]:
from TorchSisso import SissoModel
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
from pltsci import whole_plot_set, half_plot_set, set_ticks, cm

whole_plot_set()
save_dir = Path(".")
save_dir.mkdir(exist_ok=True)

In [ ]:
df=pd.read_csv("../data/TorchSISSOdf.csv")

# 合并df中完全相同的列，只保留一列
df = df.loc[:, ~df.T.duplicated()]
y=df["y"]

df_features= pd.read_csv("featurize/featurize.csv")
features_RFE=pd.read_csv("featurize/RFE_MSE.csv")
best_features=features_RFE["Features"][0]
best_features=json.loads(best_features.replace("'",'"'))
df_features=df_features[best_features]

df_features.to_csv(save_dir / "df_features无重复.csv", index=False)

df_train=pd.concat([y, df_features], axis=1)
columns=df_train.columns
df_train.columns=["y", *[f"X{i}" for i in range(1, len(columns))]]

df_train

In [ ]:
# 创建元素到原子序数的映射字典
element_to_atomic_number = {
    "H": 1,
    "He": 2,
    "Li": 3,
    "Be": 4,
    "B": 5,
    "C": 6,
    "N": 7,
    "O": 8,
    "F": 9,
    "Ne": 10,
    "Na": 11,
    "Mg": 12,
    "Al": 13,
    "Si": 14,
    "P": 15,
    "S": 16,
    "Cl": 17,
    "Ar": 18,
    "K": 19,
    "Ca": 20,
    "Sc": 21,
    "Ti": 22,
    "V": 23,
    "Cr": 24,
    "Mn": 25,
    "Fe": 26,
    "Co": 27,
    "Ni": 28,
    "Cu": 29,
    "Zn": 30,
    "Ga": 31,
    "Ge": 32,
    "As": 33,
    "Se": 34,
    "Br": 35,
    "Kr": 36,
    "Rb": 37,
    "Sr": 38,
    "Y": 39,
    "Zr": 40,
    "Nb": 41,
    "Mo": 42,
    "Tc": 43,
    "Ru": 44,
    "Rh": 45,
    "Pd": 46,
    "Ag": 47,
    "Cd": 48,
    "In": 49,
    "Sn": 50,
    "Sb": 51,
    "Te": 52,
    "I": 53,
    "Xe": 54,
    "Cs": 55,
    "Ba": 56,
    "La": 57,
    "Ce": 58,
    "Pr": 59,
    "Nd": 60,
    "Pm": 61,
    "Sm": 62,
    "Eu": 63,
    "Gd": 64,
    "Tb": 65,
    "Dy": 66,
    "Ho": 67,
    "Er": 68,
    "Tm": 69,
    "Yb": 70,
    "Lu": 71,
    "Hf": 72,
    "Ta": 73,
    "W": 74,
    "Re": 75,
    "Os": 76,
    "Ir": 77,
    "Pt": 78,
    "Au": 79,
    "Hg": 80,
    "Tl": 81,
    "Pb": 82,
    "Bi": 83,
    "Po": 84,
    "At": 85,
    "Rn": 86,
    "Fr": 87,
    "Ra": 88,
    "Ac": 89,
    "Th": 90,
    "Pa": 91,
    "U": 92,
    "Np": 93,
    "Pu": 94,
    "Am": 95,
    "Cm": 96,
    "Bk": 97,
    "Cf": 98,
    "Es": 99,
    "Fm": 100,
}


In [ ]:
df_output = pd.concat([df, df_features], axis=1)
df_output["x"] = df_output["x"].apply(lambda x: x.split("1")[0])
df_output["atomic_numbers"] = df_output["x"].map(element_to_atomic_number)
df_output.sort_values(by="atomic_numbers", inplace=True)
df_output.to_csv(save_dir / "df_output.csv", index=False)

In [ ]:
# 划分训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns=["y"]), df_train["y"], test_size=0.2, random_state=42)
df_train = pd.concat([y_train, X_train], axis=1)
df_test = pd.concat([y_test, X_test], axis=1)
df_train.to_csv(save_dir / "train.csv", index=True)
df_test.to_csv(save_dir / "test.csv", index=True)
df_train

In [ ]:
df_train = pd.read_csv(save_dir / "train.csv", index_col=0)
df_test = pd.read_csv(save_dir / "test.csv",index_col=0)
df_train_copy = df_train.copy()
df_train.reset_index(drop=True, inplace=True)
operators = ["+", "-", "*", "/", "exp", "ln", "sin", "pow(2)"]
sm = SissoModel(
    df_train,  # Takes the dataframe as input with first variable as target variable
    operators,  # Takes the user-defined operators to perform the feature engineering
    n_expansion = 3, #Defines the number of feature expansions need to be considered
    n_term=1,  # Defines the number of terms to be considered in the final model
    use_gpu=True,  # Defines the flag whether to consider GPU or not (For efficient computation we consider using GPU only for $L_0$ Regularization.
)
fit_output = sm.fit()
fit_output

In [ ]:
for i,item in enumerate(df_features.columns,start=1):
        print(f"X{i}:", item)

In [ ]:
f1, = fit_output[3]

In [ ]:
from math import log, e, exp


def ln(x):
    return log(x, e) if x > 0 else float("-inf")  # Define natural logarithm function


def predict(x: pd.Series, 表达式: str = None) -> float:
    """
    Predict the formation energy for a given input feature vector.
    :param x: pd.Series, input feature vector
    :return: float, predicted formation energy
    """
    code = "from math import *\n"
    # code = ""
    for name, value in x.items():
        code += f"{name} = {value}\n"
    code += f"result = {表达式}\n"
    local_vars = {}
    # print(code)
    exec(code, globals(), local_vars)
    return local_vars["result"]

In [ ]:
train_predict = {}
test_predict = {}
for i,f in enumerate((f1,),start=1):
    train_predict[i] = df_train_copy.apply(lambda x: predict(x, f), axis=1)
    test_predict[i] = df_test.apply(lambda x: predict(x, f), axis=1)
train_predict = pd.DataFrame(train_predict)
test_predict = pd.DataFrame(test_predict)
train_predict["y"] = df[df.index.isin(train_predict.index)]["y"]
test_predict["y"] = df[df.index.isin(test_predict.index)]["y"]
train_predict.to_csv(save_dir / "train_predict.csv", index=False)
test_predict.to_csv(save_dir / "test_predict.csv", index=False)
train_predict

In [ ]:
fig, ax = plt.subplots(figsize=(cm(7), cm(6)), dpi=600)
# ax = axs[0]
half_plot_set(ax)
# 画一条对角线
ax.plot([-4, 3], [-4, 3], color="#BE7A0D", linestyle="--", zorder=1, linewidth=0.7)
ax.scatter(train_predict["y"] , train_predict[1], label="Train", color="#5C6CE7", alpha=1, marker="o", s=5, zorder=2)
ax.scatter(test_predict["y"] , test_predict[1], label="Test", color="#C42C6E", alpha=1, marker="D", s=5, zorder=2)
set_ticks(ax, (-4, 3, 1), (-4, 3, 1))
# ax.set_xticks(np.arange(-8, 8.1 ,4))
# ax.set_yticks(np.arange(-8, 8.1 ,4))
ax.grid(which="major", linestyle="--", linewidth=0.5, alpha=0.5)
ax.set_xlabel(r"$E^{pt}$ (DFT) (eV/f.u.)", fontsize=8)
ax.set_ylabel(r"$E^{pt}$ (TorchSISSO) (eV/f.u.)", fontsize=8)

ax.legend(
    loc="upper left",
    fontsize=8,
    frameon=False,
    handlelength=1.5,
    handletextpad=0.5,
    borderpad=0.2,
    labelspacing=0.2,
)
ax.text(-1.9, 2.25, r"$R^2$ = 0.92", fontsize=8, ha="left", va="bottom")
ax.text(-1.9, 1.71, r"$R^2$ = 0.91", fontsize=8, ha="left", va="bottom")
plt.tight_layout()

In [ ]:
fig.savefig(save_dir / "TorchSISSO.png", dpi=1200, bbox_inches="tight")
fig.savefig(save_dir / "TorchSISSO.jpg", dpi=1200, bbox_inches="tight")
fig.savefig(save_dir / "TorchSISSO.pdf", dpi=1200, bbox_inches="tight")
fig.savefig(save_dir / "TorchSISSO.svg",bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(cm(16), cm(14)), dpi=300)
# for i,x in enumerate(["X4","X9","X13","X18"]):
xlabels = ["Covalent radius (pm)", "Electronegativity", "DFT volume (Å$^3$)", "Melting point (K)"]

colors = ["#BA622F", "#3545BA", "#2FA35A", "#239EA9"]
vlines = [121.0, 1.61, 16.48, 933.47]
yrange = (-4, 3, 1)
xranges = [(90, 230, 20), (0.6, 2.6, 0.2), (0, 100, 10), (0, 4000, 500)]
标注 = ["(a)", "(b)", "(c)", "(d)"]
# for i, x in enumerate([4, 10, 12, 18]):
for i, x in enumerate([0, 1, 2, 3]):
    ax = axs[i // 2, i % 2]
    half_plot_set(ax)
    ax.scatter(df_output[df_output.columns[x + 2]], df_output["y"], color="#FFFFFF")
    for id, row in df_output.iterrows():
        ax.text(
            row[df_output.columns[x + 2]],
            row["y"],
            row["x"],
            ha="center",
            va="center",
            fontsize=7,
            # color=plt.cm.viridis((row["E_pt"] - (-6)) / (6 - (-6))),
            # color=colors[i]
            color=colors[0] if row["y"] > 0 else colors[1],
        )
    set_ticks(ax, xranges[i], yrange)
    set_ticks(ax, None, yrange)
    ax.axhline(0, color="black", linestyle="--", linewidth=0.7)
    ax.axvline(vlines[i], color="black", linestyle="--", linewidth=0.7)
    # ax.set_ylabel("Phase transformation energy (eV)", fontsize=10)
    ax.set_ylabel(r"$E^{pt}$ (eV/f.u.)", fontsize=10)
    # ax.set_ylabel(r"phase transformation energy (eV)", fontsize=10)
    # ax.set_xlabel(df_output.columns[x + 2], fontsize=10)
    ax.set_xlabel(xlabels[i], fontsize=10)
    R = np.corrcoef(df_output[df_output.columns[x + 2]], df_output["y"])[0, 1]
    # R = 0
    R2 = R**2
    ax.text(
        0.75 if i % 2 == 0 else 0.03,
        0.96,
        f"$R$ = {R:.2f}",
        fontsize=10,
        transform=ax.transAxes,
        verticalalignment="top",
        # ha="left" if i % 2 == 0 else "right",
    )
    ax.grid(which="both", linestyle="--", linewidth=0.5, alpha=0.5)
    ax.text(-0.15, 1, 标注[i], fontsize=12, transform=ax.transAxes, verticalalignment="top")
    
plt.tight_layout()

In [ ]:
fig.savefig(save_dir / "feature_correlation.png", dpi=1200, bbox_inches="tight")
fig.savefig(save_dir / "feature_correlation.jpg", dpi=1200, bbox_inches="tight")
fig.savefig(save_dir / "feature_correlation.pdf", dpi=1200, bbox_inches="tight")